# Text generation using RNN - Character Level

To generate text using RNN, we need a to convert raw text to a supervised learning problem format.

Take, for example, the following corpus:

"Her brother shook his head incredulously"

First we need to divide the data into tabular format containing input (X) and output (y) sequences. In case of a character level model, the X and y will look like this:

|      X     |  Y  |
|------------|-----|
|    Her b   |  r  |
|    er br   |  o  |
|    r bro   |  t  |
|     brot   |  h  |
|    broth   |  e  |
|    .....   |  .  |
|    .....   |  .  |
|    ulous   |  l  |
|    lousl   |  y  |

Note that in the above problem, the sequence length of X is five characters and that of y is one character. Hence, this is a many-to-one architecture. We can, however, change the number of input characters to any number of characters depending on the type of problem.

A model is trained on such data. To generate text, we simply give the model any five characters using which it predicts the next character. Then it appends the predicted character to the input sequence (on the extreme right of the sequence) and discards the first character (character on extreme left of the sequence). Then it predicts again using the new sequence and the cycle continues until a fix number of iterations. An example is shown below:

Seed text: "incre"

|      X                                            |  Y                       |
|---------------------------------------------------|--------------------------|
|                        incre                      |    < predicted char 1 >  |
|               ncre < predicted char 1 >              |    < predicted char 2 >  |
|       cre< predicted char 1 > < predicted char 2 >   |    < predicted char 3 >  |
|       re< predicted char 1 >< predicted char 2 > < predicted char 3 >   |    < predicted char 4 >  |
|                      ...                          |            ...           |

# Notebook Overview
1. Preprocess data
2. LSTM model
3. Generate code

In [2]:
# import libraries
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import random
import sys
import io
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import get_file

# 1. Preprocess data

We're going to build a C code generator by training an RNN on a huge corpus of C code (the linux kernel code). You can download the C code used as source text from the following link:
https://github.com/torvalds/linux/tree/master/kernel

We have already downloaded the entire kernel folder and stored in a local directory

## Load C code

In [4]:
# set path where C files reside

path = r"kernel"

os.chdir(path)

file_names = os.listdir()
print(file_names)

['.gitignore', 'acct.c', 'async.c', 'audit.c', 'audit.h', 'auditfilter.c', 'auditsc.c', 'audit_fsnotify.c', 'audit_tree.c', 'audit_watch.c', 'backtracetest.c', 'bounds.c', 'bpf', 'capability.c', 'cfi.c', 'cgroup', 'compat.c', 'configs', 'configs.c', 'context_tracking.c', 'cpu.c', 'cpu_pm.c', 'crash_core.c', 'crash_dump_dm_crypt.c', 'crash_reserve.c', 'cred.c', 'debug', 'delayacct.c', 'dma', 'dma.c', 'elfcorehdr.c', 'entry', 'events', 'exec_domain.c', 'exit.c', 'exit.h', 'extable.c', 'fail_function.c', 'fork.c', 'freezer.c', 'futex', 'gcov', 'gen_kheaders.sh', 'groups.c', 'hung_task.c', 'iomem.c', 'irq', 'irq_work.c', 'jump_label.c', 'kallsyms.c', 'kallsyms_internal.h', 'kallsyms_selftest.c', 'kallsyms_selftest.h', 'kcmp.c', 'Kconfig.freezer', 'Kconfig.hz', 'Kconfig.kexec', 'Kconfig.locks', 'Kconfig.preempt', 'kcov.c', 'kcsan', 'kexec.c', 'kexec_core.c', 'kexec_elf.c', 'kexec_file.c', 'kexec_handover.c', 'kexec_internal.h', 'kheaders.c', 'kprobes.c', 'ksyms_common.c', 'ksysfs.c', 'kthre

In [5]:
# use regex to filter .c files
import re
c_names = ".*\.c$"

c_files = list()

for file in file_names:
    if re.match(c_names, file):
        c_files.append(file)

print(c_files)

['acct.c', 'async.c', 'audit.c', 'auditfilter.c', 'auditsc.c', 'audit_fsnotify.c', 'audit_tree.c', 'audit_watch.c', 'backtracetest.c', 'bounds.c', 'capability.c', 'cfi.c', 'compat.c', 'configs.c', 'context_tracking.c', 'cpu.c', 'cpu_pm.c', 'crash_core.c', 'crash_dump_dm_crypt.c', 'crash_reserve.c', 'cred.c', 'delayacct.c', 'dma.c', 'elfcorehdr.c', 'exec_domain.c', 'exit.c', 'extable.c', 'fail_function.c', 'fork.c', 'freezer.c', 'groups.c', 'hung_task.c', 'iomem.c', 'irq_work.c', 'jump_label.c', 'kallsyms.c', 'kallsyms_selftest.c', 'kcmp.c', 'kcov.c', 'kexec.c', 'kexec_core.c', 'kexec_elf.c', 'kexec_file.c', 'kexec_handover.c', 'kheaders.c', 'kprobes.c', 'ksyms_common.c', 'ksysfs.c', 'kthread.c', 'latencytop.c', 'module_signature.c', 'notifier.c', 'nsproxy.c', 'padata.c', 'panic.c', 'params.c', 'pid.c', 'pid_namespace.c', 'profile.c', 'ptrace.c', 'range.c', 'reboot.c', 'regset.c', 'relay.c', 'resource.c', 'resource_kunit.c', 'rseq.c', 'scftorture.c', 'scs.c', 'seccomp.c', 'signal.c', 's

In [6]:
# load all c code in a list
full_code = list()
for file in c_files:
    code = open(file, "r", encoding='utf-8')
    full_code.append(code.read())
    code.close()

In [7]:
# let's look at how a typical C code looks like
print(full_code[20])

// SPDX-License-Identifier: GPL-2.0-or-later
/* Task credentials management - see Documentation/security/credentials.rst
 *
 * Copyright (C) 2008 Red Hat, Inc. All Rights Reserved.
 * Written by David Howells (dhowells@redhat.com)
 */

#define pr_fmt(fmt) "CRED: " fmt

#include <linux/export.h>
#include <linux/cred.h>
#include <linux/slab.h>
#include <linux/sched.h>
#include <linux/sched/coredump.h>
#include <linux/key.h>
#include <linux/keyctl.h>
#include <linux/init_task.h>
#include <linux/security.h>
#include <linux/binfmts.h>
#include <linux/cn_proc.h>
#include <linux/uidgid.h>

#if 0
#define kdebug(FMT, ...)						\
	printk("[%-5.5s%5u] " FMT "\n",					\
	       current->comm, current->pid, ##__VA_ARGS__)
#else
#define kdebug(FMT, ...)						\
do {									\
	if (0)								\
		no_printk("[%-5.5s%5u] " FMT "\n",			\
			  current->comm, current->pid, ##__VA_ARGS__);	\
} while (0)
#endif

static struct kmem_cache *cred_jar;

/* init to 2 - one for init_task, one to ensure it is never f

In [8]:
# merge different c codes into one big c code
text = "\n".join(full_code)
print("Total number of characters in entire code: {}".format(len(text)))

Total number of characters in entire code: 2276960


In [9]:
# top_n: only consider first top_n characters and discard the rest for memory and computational efficiency
top_n = 400000
text = text[:top_n]

## Convert characters to integers

In [10]:
# create character to index mapping
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [11]:
print("Vocabulary size: {}".format(len(chars)))

Vocabulary size: 95


## Divide data in input (X) and output (y)

### Create sequences

In [12]:
# define length for each sequence
MAX_SEQ_LENGTH = 50          # number of input characters (X) in each sequence 
STEP           = 3           # increment between each sequence
VOCAB_SIZE     = len(chars)  # total number of unique characters in dataset

sentences  = []              # X
next_chars = []              # y

for i in range(0, len(text) - MAX_SEQ_LENGTH, STEP):
    sentences.append(text[i: i + MAX_SEQ_LENGTH])
    next_chars.append(text[i + MAX_SEQ_LENGTH])

In [13]:
print('Number of training samples: {}'.format(len(sentences)))

Number of training samples: 133317


## Create input and output using the created sequences

When you're not using the Embedding layer of the Keras as the very first layer, you need to convert your data in the following format:
#### input shape should be of the form :  (#samples, #timesteps, #features)
#### output shape should be of the form :  (#samples, #timesteps, #features)

![Tensor shape](./jupyter resources/rnn_tensor.png)

#samples: the number of data points (or sequences)
#timesteps: It's the length of the sequence of your data (the MAX_SEQ_LENGTH variable).
#features: Number of features depends on the type of problem. In this problem, #features is the vocabulary size, that is, the dimensionality of the one-hot encoding matrix using which each character is being represented. If you're working with **images**, features size will be equal to: (height, width, channels), and the input shape will be (#training_samples, #timesteps, height, width, channels)

In [15]:
# create X and y
X = np.zeros((len(sentences), MAX_SEQ_LENGTH, VOCAB_SIZE), dtype=bool)
y = np.zeros((len(sentences), VOCAB_SIZE), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [16]:
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (133317, 50, 95)
Shape of y: (133317, 95)


Here, X is reshaped to (#samples, #timesteps, #features). We have explicitly mentioned the third dimension (#features) because we won't use the Embedding() layer of Keras in this case since there are only 97 characters. Characters can be represented as one-hot encoded vector. There are no word embeddings for characters.

# 2. LSTM

In [31]:
# define model architecture - using a two-layer LSTM with 128 LSTM cells in each layer
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_SEQ_LENGTH, VOCAB_SIZE), return_sequences=True, dropout=0.3))
model.add(LSTM(128, dropout=0.3))
model.add(Dense(VOCAB_SIZE, activation = "softmax"))

optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['acc'])

In [24]:
# check model summary
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                        │ (None, 128)                 │         114,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 95)                  │          12,255 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 126,943 (495.87 KB)

 Trainable params: 126,943 (495.87 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# fit model
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/20
 140/1042 ━━━━━━━━━━━━━━━━━━━━ 2:11 146ms/step - acc: 0.5219 - loss: 1.7237

KeyboardInterrupt: 

# 3. Generate code

Create a function that will make next character predictions based on temperature. If temperature is greater than 1, the generated characters will be more versatile and diverse. On the other hand, if temperature is less than one, the generated characters will be much more conservative.

In [27]:
# define function to sample next word from a probability array based on temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [28]:
np.random.multinomial(10, [0.05, 0.9, 0.05], size=2)

array([[ 0, 10,  0],
       [ 0, 10,  0]])

In [29]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random code to start text generation

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "ts that match */
				if (!ctx || ctx->type != AUDI"
ts that match */
T_MAX; (!ctx || ctx->type != AUDI
				return -EINVAL;
				return -EINVAL;
		}
			}
			ret = audit_log_enum_cpuhp_cpu_astath(struct audit_buffer *ab, struct audit_chunk *parent);

static int audit_context = par_cpu_unlock();

	return -EINVAL;
	}
	}

																		 from_kurid(&context->count);
										                                                                                                                                                                                                                                                                                                                                                                  struct audit real read audit_context);

static void audit_context = audit_comperdt(audit_callby);
		if (err ==
														TARN_IN_RE_DECALED)
			if (!old->list[lent) {
				

In [30]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random seed

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "		audit_log_format(ab, "nargs=%d", nargs);
		for ("
		audit_log_format(ab, "nargs=%d", nargs);
rev = 0;
													              const char *ab)
{
	struct audit_context *context = audit_net_unlock();
				}
			if (f->op, AUDIT_IORED)
		return -EINVAL;
	}

	/* This backle bool the seepe all audit message to the nase oll in income it async_synchronization by ingup. */
				}
				return -EINVAL;
}

static int audit_context = audit_comparator(tree);
				err = audit_log_n_enter(tree;

		if (!context->st->state == AUDIT_TOM_UDIT_ALDDD)
				/* take the called to the context the canly */
				if (!err == AUDIT_STATE_RCEL,
							              forms > nert)
																                  entry->rule.lock < AUDIT_TRACELENETERD && errer)
{
	struct audit_context *context = audit_filter_mutex);

	return 0;
}

itt audit_filter_user(struct audit_context *ctx,
			                                           